In [7]:
import cv2
import os
import pandas as pd
import Augmentor
import numpy as np
import seabo##rn as sns

# Train

In [8]:
data=[]
labels = []
list_classes =[ 0, 1, 14, 13, 33 ,34]
classes = 43 # len(list_classes)
tam_img = 50
n_colour = 3
cur_path = os.getcwd()
list_classes

[0, 1, 14, 13, 33, 34]

In [9]:
os.path.exists(cur_path)

True

In [10]:
for i in range(43) : #list_classes:
    path = os.path.join(cur_path,'Train',str(i))
    images = os.listdir(path)
    for img in images:
        img2=img
        try:
            img_path = os.path.join(path, img)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (tam_img,tam_img), interpolation = cv2.INTER_CUBIC)/255
            data.append(img)
            labels.append(i)
        except:
            print(f"Error al leer {img2} {i}")

In [11]:
np.amax(data[0])

1.0

In [12]:
train = pd.DataFrame(columns= ["label", "image"])
train["label"] = labels
train["image"] = data

In [13]:
train.head()

,label,image
0,0,"[[[0.44313725490196076, 0.4, 0.360784313725490..."
1,0,"[[[0.6470588235294118, 0.3176470588235294, 0.3..."
2,0,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
3,0,"[[[0.6745098039215687, 0.1450980392156863, 0.1..."
4,0,"[[[0.7019607843137254, 0.6352941176470588, 0.6..."


In [14]:
train.shape

(39209, 2)

In [15]:
train["label"].value_counts()

2     2250
1     2220
13    2160
12    2100
38    2070
10    2010
4     1980
5     1860
25    1500
9     1470
7     1440
3     1410
8     1410
11    1320
35    1200
18    1200
17    1110
31     780
14     780
33     689
15     630
26     600
28     540
23     510
30     450
16     420
34     420
6      420
36     390
22     390
40     360
20     360
21     330
39     300
29     270
24     270
41     240
42     240
32     240
27     240
37     210
19     210
0      210
Name: label, dtype: int64

## Data Augmentation

In [ ]:
for label,value in zip(train["label"].value_counts().index, train["label"].value_counts().values) :
    if 2000>value:
        print(label, value)
        path = os.path.join(cur_path,'Train',str(label), 'output')
        if not os.path.exists(path):
            print("Creacion de carpeta output")
            p = Augmentor.Pipeline(os.path.join(cur_path,'Train',str(label)))
            p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
            p.zoom(probability=0.3, min_factor=1.1, max_factor=1.6)
            p.skew(probability=0.6)
            p.sample(1000-value)  
        
 
        images = os.listdir(path)
        for img in images:
            img2=img
            try:
                img_path = os.path.join(path, img)
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (tam_img,tam_img), interpolation = cv2.INTER_CUBIC)/255
                data.append(img)
                labels.append(label)
            except:
                print(f"Error al leer {img2}")
        
        


In [ ]:
train = pd.DataFrame(columns= ["label", "image"])
train["label"] = labels
train["image"] = data 
train["label"].value_counts()

In [ ]:
# 13 -> 2
# 14 -> 3
# 33 -> 4
# 34 -> 5

train["label"] = train["label"].replace([13],2)
train["label"] = train["label"].replace([14],3)
train["label"] = train["label"].replace([33],4)
train["label"] = train["label"].replace([34],5)

In [ ]:
train.shape

In [ ]:
data_train = np.array(train["image"].tolist())

In [ ]:
data_train.shape

In [ ]:
data_train.shape

# Test

In [ ]:
test = pd.read_csv( 'Test.csv')

labels = test["ClassId"].values



data =[]
path = os.path.join(cur_path,'Test')
images = os.listdir(path)

for img in images:
        img2= img
        try:
            img_path = os.path.join(path, img)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (tam_img,tam_img), interpolation = cv2.INTER_CUBIC)/255
            data.append(img)
        except:
            print(f"Error al leer {img2}")








In [ ]:
test = pd.DataFrame(columns= ["label", "image"])
test["label"] = labels
test["image"] = data

In [ ]:
test = test[test["label"].isin(list_classes)]

In [ ]:
test["label"].value_counts()

In [ ]:
# 13 -> 2
# 14 -> 3
# 33 -> 4
# 34 -> 5

test["label"] = test["label"].replace([13],2)
test["label"] = test["label"].replace([14],3)
test["label"] = test["label"].replace([33],4)
test["label"] = test["label"].replace([34],5)

In [ ]:
data_test = np.array(test["image"].tolist())

# Modelo 

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical

In [ ]:
#Avoid OOM errors by setting PGU Memory Consumption Growth
#gpus = tf.config.experimental.list_physical_devices('GPU')
#for gpu in gpus:
 #   tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
#Converting the labels into one hot encoding
y_train = to_categorical(train["label"].values, classes)
y_test = to_categorical(test["label"].values, classes)

# VGG MODEL

In [ ]:
vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(tam_img,tam_img,n_colour))
vgg19.summary()

In [ ]:
vgg19.layers[-1].output_shape[1:]

In [ ]:
vgg_model = tf.keras.Sequential([VGG19(weights='imagenet', include_top=False, 
                                       input_shape=(tam_img,tam_img,n_colour)),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Flatten(),
                                 keras.layers.Dense(128, activation='sigmoid'),
                                 keras.layers.Dense(classes, activation='softmax')
                                ])
vgg_model.summary()

In [ ]:
alpha=0.001
epochs=4
optim = keras.optimizers.Adam(learning_rate=alpha)
vgg_model.compile(optimizer = optim, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
y_test.shape, data_test.shape, y_train.shape, data_test.shape

In [ ]:
type(y_test), type(data_test), type(y_train), type(data_train)

In [ ]:
#callbacks
Stoppincallback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', 
                                min_delta=0.05, 
                                patience=2, 
                                verbose=0, 
                                mode='auto', 
                                baseline=None, 
                                restore_best_weights=True)


Checkpointcallback = tf.keras.callbacks.ModelCheckpoint("model_checkpoint.h5", 
                                     monitor='loss', 
                                     verbose=0, 
                                     save_best_only=False,
                                     save_weights_only=False, 
                                     save_freq='epoch')

tbCallBack = keras.callbacks.TensorBoard(log_dir="Tensorboard", histogram_freq=0,
                                         write_graph=True, write_images=False)

CSVcallback = tf.keras.callbacks.CSVLogger("model_csv.csv", 
                             separator=',', 
                             append=False)                        

In [ ]:
len(data_train)

In [ ]:
"""
Epoch 1/10
387/387 [==============================] - 2328s 6s/step - loss: 0.9820 - accuracy: 0.5460 - val_loss: 3.3631 - val_accuracy: 0.2933
Epoch 2/10
387/387 [==============================] - 2118s 5s/step - loss: 0.6672 - accuracy: 0.6781 - val_loss: 4.8172 - val_accuracy: 0.1710
Epoch 3/10
387/387 [==============================] - 1751s 5s/step - loss: 0.5467 - accuracy: 0.7633 - val_loss: 4.5646 - val_accuracy: 0.2233
Epoch 4/10
199/387 [==============>...............] - ETA: 12:39 - loss: 0.3542 - accuracy: 0.8660
"""

In [ ]:
vgg_model.fit(data_train, y_train,
                        epochs=15, callbacks =[Stoppincallback,
                                               Checkpointcallback,
                                              tbCallBack, CSVcallback], 
              validation_data=(data_test, y_test)
                        )


In [ ]:
vgg_model.save("my_model_vgg_mitad.h5")

In [ ]:
model.save_weights("model_vgg_mitadweights.h5")

# Mobilenet

In [ ]:
from tensorflow.keras.applications import MobileNetV3Small

In [ ]:
base_model = MobileNetV3Small(weights="imagenet", 
                              input_shape=(tam_img,tam_img,n_colour),
                              include_top=False)

x= base_model.output
x= keras.layers.GlobalAveragePooling2D()(x)
x= keras.layers.Dense(512, activation ='relu')(x)
x= keras.layers.Dense(256, activation ='relu')(x)
x= keras.layers.Dense(128, activation ='relu')(x)
preds = keras.layers.Dense(classes, activation ='softmax')(x)

model = keras.Model(inputs=base_model.input, outputs=preds)

In [ ]:
model.summary()

In [ ]:
#for layers in model.layers[:-5]:
 #   layers.trainable=False

In [ ]:
model.summary()

In [ ]:
alpha=0.001
optim = keras.optimizers.Adam(learning_rate=alpha)
model.compile(optimizer = optim, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
#callbacks
Stoppincallback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', 
                                min_delta=0.05, 
                                patience=2, 
                                verbose=0, 
                                mode='auto', 
                                baseline=None, 
                                restore_best_weights=True)


Checkpointcallback = tf.keras.callbacks.ModelCheckpoint("model_mobilenet.h5", 
                                     monitor='loss', 
                                     verbose=0, 
                                     save_best_only=False,
                                     save_weights_only=False, 
                                     save_freq='epoch')

tbCallBack = keras.callbacks.TensorBoard(log_dir="Tensorboard", histogram_freq=0,
                                         write_graph=True, write_images=False)

CSVcallback = tf.keras.callbacks.CSVLogger("model_mobilenet.csv", 
                             separator=',', 
                             append=False)   

In [ ]:
model.fit(data_train, y_train,
                        epochs=15, callbacks =[Stoppincallback,
                                               Checkpointcallback,
                                              tbCallBack, CSVcallback], validation_split=0.30,
              #validation_data=(data_test, y_test)
                        )